In [2]:
!pip install -U pvlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 29.0 MB/s eta 0:00:00


In [4]:
import pvlib
from pvlib import pvsystem
import numpy as np
import pandas as pd
import time
import plotly.graph_objects as go

# --- Função para extrair parâmetros via De Soto
def desotorefparameters(Vmp, Imp, Voc, Isc, alpha, beta, Ns, EgRef, dEgdT, Tref, Gref):
    desoto_fit_params = pvlib.ivtools.sdm.fit_desoto(
        Vmp, Imp, Voc, Isc, alpha, beta, Ns, EgRef, dEgdT, Tref, Gref,
        root_kwargs={'method': 'lm', 'options': {'maxiter': 10000, 'xtol': 1e-3, 'gtol': 1e-3}})
    fitted_params = desoto_fit_params[0]
    return pd.DataFrame([fitted_params], columns=['a_ref', 'I_L_ref', 'I_o_ref', 'R_sh_ref', 'R_s'])

# --- Função para gerar curva IV com método Lambert W
def desotoparametersopc(irrad, temp_cell, alpha, a_ref, Ipv_ref, Io_ref, Rp_ref, R_s,
                        EgRef, dEgdT, Gref, Tref, method='lambertw', number_of_points=100):
    diode_params = pvlib.pvsystem.calcparams_desoto(
        irrad, temp_cell, alpha, a_ref, Ipv_ref, Io_ref, Rp_ref, R_s, EgRef, dEgdT, Gref, Tref)
    SDE_params = {
        'photocurrent': diode_params[0],
        'saturation_current': diode_params[1],
        'resistance_series': diode_params[2],
        'resistance_shunt': diode_params[3],
        'nNsVth': diode_params[4]
    }
    curve_info = pvsystem.singlediode(method=method, **SDE_params)
    v = np.linspace(0, curve_info['v_oc'], number_of_points)
    i = pvsystem.i_from_v(voltage=v, method=method, **SDE_params)
    return {'Key_Points': curve_info, 'Voltage': v, 'Current': i}

# --- Constantes e parâmetros do módulo
EgRef = 1.121
dEgdT = -0.0002677
Tref = 25
Gref = 1000
Vmp = 30.8
Imp = 8.75
Voc = 37.9
Isc = 9.32
alpha = 0.05 * Isc / 100
beta = (-0.31) * Voc / 100
Ns = 60
irrad = 1000
temp_cell = 25

# --- Extração de parâmetros SDM
params = desotorefparameters(Vmp, Imp, Voc, Isc, alpha, beta, Ns,
                              EgRef, dEgdT, Tref, Gref).iloc[0]
results = desotoparametersopc(irrad, temp_cell, alpha,
                              params['a_ref'], params['I_L_ref'], params['I_o_ref'],
                              params['R_sh_ref'], params['R_s'],
                              EgRef, dEgdT, Gref, Tref)

V = results['Voltage']
I = results['Current']
P = V * I
Vmp_model = results['Key_Points']['v_mp']
Pmp_model = results['Key_Points']['p_mp']

# --- Algoritmo P&O com passo fixo de 0.25 V
start_po = time.time()
deltaV = 0.25
V_curr = V[-1]
P_curr = P[-1]
V_mppt_po = [V_curr]
P_mppt_po = [P_curr]

for _ in range(50):
    V_next = V_curr + deltaV
    if V_next < V[0] or V_next > V[-1]:
        deltaV = -deltaV
        continue

    P_next = np.interp(V_next, V, P)
    if P_next > P_curr:
        V_curr = V_next
        P_curr = P_next
    else:
        deltaV = -deltaV
        V_curr += deltaV
        P_curr = np.interp(V_curr, V, P)

    V_mppt_po.append(V_curr)
    P_mppt_po.append(P_curr)

end_po = time.time()

# --- Tensão Fixa
V_fix = 30.0
I_fix = np.interp(V_fix, V, I)
P_fix = V_fix * I_fix

# --- Resultados
potencias = {
    'P&O': P_mppt_po[-1],
    'Tensão Fixa (30V)': P_fix
}
iteracoes = {
    'P&O': len(P_mppt_po),
    'Tensão Fixa (30V)': 1
}
tempos = {
    'P&O': end_po - start_po,
    'Tensão Fixa (30V)': 0
}
desvios = {k: 100 * abs(v - Pmp_model) / Pmp_model for k, v in potencias.items()}
eficiencias = {k: 100 * v / Pmp_model for k, v in potencias.items()}

df_resultados = pd.DataFrame({
    'Potência (W)': potencias,
    'Iterações': iteracoes,
    'Tempo (s)': tempos,
    'Desvio % MPP': desvios,
    'Eficiência MPPT (%)': eficiencias
}).round(4)

print(df_resultados)

# --- Gráfico interativo com Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=V, y=P, mode='lines', name='Curva P-V (modelo De Soto)',
                         line=dict(color='gray', width=2)))

fig.add_trace(go.Scatter(x=V_mppt_po, y=P_mppt_po, mode='markers+lines', name='P&O',
                         marker=dict(color='red', size=6), line=dict(dash='dot')))

fig.add_trace(go.Scatter(x=[V_fix], y=[P_fix], mode='markers', name='Tensão Fixa (30 V)',
                         marker=dict(color='green', size=10, symbol='square')))

fig.add_trace(go.Scatter(x=[Vmp_model], y=[Pmp_model], mode='markers', name='Ponto MPP Modelo',
                         marker=dict(color='black', size=10, symbol='x')))

fig.update_layout(
    title='Comparação entre Algoritmos MPPT',
    xaxis_title='Tensão (V)',
    yaxis_title='Potência (W)',
    template='plotly_white',
    hovermode='closest',
    legend=dict(x=0.01, y=0.99),
    width=900,
    height=500
)

fig.show()


                   Potência (W)  Iterações  Tempo (s)  Desvio % MPP  \
P&O                    269.4216         50     0.0025        0.0306   
Tensão Fixa (30V)      267.9143          1     0.0000        0.5899   

                   Eficiência MPPT (%)  
P&O                            99.9694  
Tensão Fixa (30V)              99.4101  
